In [10]:
import os
import json
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai
from semantic_kernel.prompt_template import PromptTemplateConfig, InputVariable
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

In [14]:
kernel = sk.Kernel()

# load some variables for azure openai from .env
load_dotenv()
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

kernel.add_service(
    AzureChatCompletion(
        service_id="gpt4o",
        deployment_name=deployment,
        endpoint=endpoint,
        api_key=api_key,
    ),
)

kernel.add_service(
    AzureChatCompletion(
        service_id="embedding",
        deployment_name="text-embedding-ada-002",
        endpoint=endpoint,
        api_key=api_key,
    ),
)

In [6]:
kernel.add_service(chat_service)


kernel.add_service(embedding_gen)

In [7]:
base_plugin = "./plugins"

In [8]:
files_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "FilePlugin")

In [9]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [10]:
kblist = []

In [11]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = await kernel.invoke(notesFunc, sk.KernelArguments(input=content))
    json_result = json.loads(str(result))
    kblist.append(json_result)
    file.close()

In [12]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = await kernel.invoke(transcripsFunc, sk.KernelArguments(input=content))
    # print(result.result)
    json_result = json.loads(str(result))
    for item in json_result:
        kblist.append(item)
    file.close()

In [13]:
kblist

[{'kb': 'History of machine learning',
  'content': "The history of machine learning and artificial intelligence is marked by notable discoveries and periods of significant progress and setbacks. Key discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). The concept of a 'machine that can think' was introduced by Alan Turing in 1950. The Dartmouth Summer Research Project in 1956 was a seminal event for AI as a field. The period from 1956 to 1974 is often referred to as 'The golden years' of AI. However, by the mid-1970s, the complexity of creating 'intelligent machines' had been understated and its promise overblown, leading to an 'AI Winter'. The 1980s saw the proliferation of 'expert systems'. The period from 1993 to 2011 saw a new era for ML and AI with the rapid increase of data and compute power. Today, machine learning and AI touch a

In [14]:
base_vectordb = 'aboutMLKBDemoDemo'

In [15]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

In [16]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection(base_vectordb)

In [17]:
memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=embedding_gen)

In [18]:
import uuid

In [19]:
uuid.uuid4()

UUID('06b7e53f-9764-404a-96df-0a68a1d8bd45')

In [20]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await memory.save_information(base_vectordb, id=id, text=content)

In [21]:
ask = "can you tell me what is different ML and AI"

memories = await memory.search(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [22]:
result = ''
for item in memories:
    print(f"Top Result: {item.text} with score {item.relevance}")
    result = item.text

Top Result: The difference between AI and ML - AI is task accomplishment, ML is AI subset, uses data learning algorithms with score 0.87940943


In [23]:
answer_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "AnswerPlugin")

In [24]:
answerFunc = answer_plugin["Summary"]

In [25]:
summary_result = await kernel.invoke(answerFunc, sk.KernelArguments(input=result))

In [26]:
str(summary_result)

'AI, or Artificial Intelligence, is about enabling machines to perform tasks that would normally require human intelligence. On the other hand, Machine Learning (ML) is a subset of AI that focuses on using data and algorithms to enable machines to learn and improve from experience. In simple terms, AI is about smart task completion, while ML is about learning from data to perform these tasks more effectively.'